# Keras Tuner | Hyperparameter Tuning a Neural Network


### Import Necessary Libraries


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras_tuner as kt
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings


warnings.filterwarnings('ignore')

### Load The Data


In [3]:
df = pd.read_csv(r'data/diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [7]:
# There is two type of datatype:
# float64(2), int64(7)

In [8]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [9]:
df.shape

(768, 9)

In [10]:
# there is:
# Row = 768
# columns = 9

In [11]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [12]:
# let's check all columns correlaton with 'Outcome'

In [13]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [14]:
# let's Check Null values
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [15]:
# there is no Null values

In [16]:
df.duplicated().value_counts()

False    768
Name: count, dtype: int64

In [17]:
# There is no duplicated

In [18]:
# let's extract the columns
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [19]:
X.shape, y.shape

((768, 8), (768,))

In [20]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [21]:
# let's scaling X data mean feature data

In [22]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [23]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [24]:
X.shape

(768, 8)

In [25]:
# lets' splitting the data

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)

# Modeling with keras Tuner


In [132]:
# Normal model witout keras_tuner
model = Sequential()

model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam',
              loss='binary_crossentropy', metrics=['accuracy'])

In [133]:
model.fit(X_train, y_train, batch_size=32,
          epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
20/20 [==============================] - 2s 19ms/step - loss: 0.6294 - accuracy: 0.6580 - val_loss: 0.6014 - val_accuracy: 0.6948
Epoch 2/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5850 - accuracy: 0.7166 - val_loss: 0.5640 - val_accuracy: 0.7532
Epoch 3/10
20/20 [==============================] - 0s 6ms/step - loss: 0.5559 - accuracy: 0.7508 - val_loss: 0.5363 - val_accuracy: 0.7532
Epoch 4/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5341 - accuracy: 0.7638 - val_loss: 0.5162 - val_accuracy: 0.7662
Epoch 5/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5185 - accuracy: 0.7606 - val_loss: 0.5015 - val_accuracy: 0.7597
Epoch 6/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5065 - accuracy: 0.7557 - val_loss: 0.4914 - val_accuracy: 0.7662
Epoch 7/10
20/20 [==============================] - 0s 6ms/step - loss: 0.4968 - accuracy: 0.7671 - val_loss: 0.4838 - val_accuracy: 0.7727
Epoch 8/10
20/20 [=

In [134]:
# val_accuracy: 0.7857

In [135]:
# let's build Model with keras_tuner

def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer',
                          values=['rmsprop', 'adam', 'sgd', 'adadelta'])
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [136]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='optimizerTuning',
)

In [137]:
tuner.search(X_train, y_train, epochs=5,
             validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.701298713684082

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 12s


In [138]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [139]:
model = tuner.get_best_models(num_models=1)[0]

In [140]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [141]:
model.fit(X_train, y_train, batch_size=32,
          epochs=100, initial_epoch=6,
          validation_data=(X_test, y_test))

Epoch 7/100
20/20 [==============================] - 2s 19ms/step - loss: 0.5280 - accuracy: 0.7410 - val_loss: 0.5091 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5137 - accuracy: 0.7557 - val_loss: 0.5005 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5039 - accuracy: 0.7541 - val_loss: 0.4922 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4948 - accuracy: 0.7557 - val_loss: 0.4872 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4875 - accuracy: 0.7606 - val_loss: 0.4840 - val_accuracy: 0.8117
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4815 - accuracy: 0.7638 - val_loss: 0.4809 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4762 - accuracy: 0.7622 - val_loss: 0.4780 - val_accuracy: 0.8182
Epoch 14

In [142]:
# After optimizer tuner val_accuracy: 0.8182

In [153]:
# let's tuning neuron
def build_model(hp):
    model = Sequential()
    # Tune the number of units in the first Dense layer
    units = hp.Int('units',
                   min_value=8, max_value=128, step=8)
    model.add(Dense(units=units,
                    activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='Adam',
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [154]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='NeuronTuning',
)

In [155]:
tuner.search(X_train, y_train, epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 15s


In [156]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [157]:
model = tuner.get_best_models(num_models=1)[0]

In [158]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 104)               936       
                                                                 
 dense_1 (Dense)             (None, 1)                 105       
                                                                 
Total params: 1041 (4.07 KB)
Trainable params: 1041 (4.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [161]:
model.fit(X_train, y_train, batch_size=32,
          epochs=100, initial_epoch=6,
          validation_data=(X_test, y_test))

Epoch 7/100
20/20 [==============================] - 0s 18ms/step - loss: 0.3518 - accuracy: 0.8404 - val_loss: 0.4659 - val_accuracy: 0.8117
Epoch 8/100
20/20 [==============================] - 0s 10ms/step - loss: 0.3499 - accuracy: 0.8404 - val_loss: 0.4658 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 14ms/step - loss: 0.3510 - accuracy: 0.8388 - val_loss: 0.4634 - val_accuracy: 0.8117
Epoch 10/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3490 - accuracy: 0.8420 - val_loss: 0.4624 - val_accuracy: 0.8182
Epoch 11/100
20/20 [==============================] - 0s 7ms/step - loss: 0.3488 - accuracy: 0.8453 - val_loss: 0.4660 - val_accuracy: 0.8182
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3484 - accuracy: 0.8485 - val_loss: 0.4674 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3504 - accuracy: 0.8453 - val_loss: 0.4614 - val_accuracy: 0.8117
Epoch 

In [162]:
# after neuron tuning it's improved have a look: val_accuracy: 0.8247

### Let's Tuning All Togather


In [107]:
def build_model(hp):
    model = keras.Sequential()

    model.add(keras.layers.Dense(units=hp.Int('units', min_value=8, max_value=128, step=8),
                                 activation='relu', input_dim=8))
    model.dropout = hp.Float('dropout', min_value=0.1, max_value=0.9, step=0.1)
    for i in range(hp.Int('num_layers', 1, 10)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                  min_value=8,
                                                  max_value=128,
                                                  step=8),
                                     activation=hp.Choice('activation_' + str(i),
                                                          values=['relu', 'tanh', 'sigmoid']),
                                     input_dim=8))
        # model.add(keras.layers.Dropout(hp.Choice('dropout' + str(i),
                                                #  values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer=(hp.Choice('optimizer',
                             values=['sgd', 'rmsprop', 'adam', 'nadam', 'adadelta'])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [108]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='FinalTune',
    project_name='Final_Tune'
)

In [109]:
tuner.search(X_train, y_train, epochs=5,
             validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 13s


In [119]:
tuner.get_best_hyperparameters()[0].values

{'units': 16,
 'dropout': 0.2,
 'num_layers': 2,
 'units_0': 48,
 'activation_0': 'tanh',
 'optimizer': 'nadam',
 'units_1': 112,
 'activation_1': 'relu',
 'units_2': 128,
 'activation_2': 'sigmoid',
 'units_3': 120,
 'activation_3': 'sigmoid'}

In [120]:
model = tuner.get_best_models(num_models=1)[0]

In [121]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                144       
                                                                 
 dense_1 (Dense)             (None, 48)                816       
                                                                 
 dense_2 (Dense)             (None, 112)               5488      
                                                                 
 dense_3 (Dense)             (None, 1)                 113       
                                                                 
Total params: 6561 (25.63 KB)
Trainable params: 6561 (25.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [122]:
model.fit(X_train, y_train, batch_size=32,
          epochs=100, initial_epoch=6,
          validation_data=(X_test, y_test))

Epoch 7/100
20/20 [==============================] - 3s 26ms/step - loss: 0.4648 - accuracy: 0.7704 - val_loss: 0.4674 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4584 - accuracy: 0.7850 - val_loss: 0.4740 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4545 - accuracy: 0.7850 - val_loss: 0.4668 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4490 - accuracy: 0.7769 - val_loss: 0.4623 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4469 - accuracy: 0.7883 - val_loss: 0.4628 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4426 - accuracy: 0.7915 - val_loss: 0.4668 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4421 - accuracy: 0.7785 - val_loss: 0.4660 - val_accuracy: 0.7922
Epoch 14